In [ ]:
# Execution Strategies Benchmark

This notebook benchmarks three execution strategies: naive Splitting (TWAP), AdaptiveSplittingBot, and GreedyAdaptiveBot across simple market regimes.

# Install & Import Dependencies
import sys
import os
from pathlib import Path
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

print('Python', sys.version.splitlines()[0])
print('NumPy', np.__version__)

ROOT = Path(os.getcwd())
ARTIFACTS = ROOT / 'artifacts'
ARTIFACTS.mkdir(exist_ok=True)

# Setup functions
from market_engine import MarketEngine
from bots import MarketMaker, SplittingBot, AdaptiveSplittingBot, GreedyAdaptiveBot


def run_strategy(engine: MarketEngine, bot, start_params: dict, max_ticks: int = 50):
    engine.register_bot(bot)
    bot.start_order(**start_params)
    for _ in range(max_ticks):
        engine.step()
        if not getattr(bot, 'active', False):
            break
    executed = sum(r['executed_size'] for r in getattr(bot, 'executions', []))
    total_cost = sum((r['vwap'] or 0.0) * r['executed_size'] for r in getattr(bot, 'executions', []))
    avg_price = total_cost / executed if executed > 0 else None
    return executed, avg_price

# Task: Benchmark across two regimes
regimes = [
    {'name': 'thin', 'initial_asks': [(105.0, 1.0), (106.0, 2.0), (107.0, 3.0)]},
    {'name': 'deep', 'initial_asks': [(102.0, 5.0), (103.0, 10.0), (104.0, 10.0)]},
]

results = []
for regime in regimes:
    for seed in [1, 2, 3]:
        rng = None
        # Splitting
        eng = MarketEngine(init_price=100.0, vol=0.0, rng=rng)
        for p, s in regime['initial_asks']:
            eng.place_order({'side': 'sell', 'price': p, 'size': s, 'bot': None})
        mm = MarketMaker(spread=1.0, size=1.0, jitter=0.0)
        eng.register_bot(mm)
        sb = SplittingBot()
        executed, avg_price = run_strategy(eng, sb, {'side':'buy', 'total_size':6.0, 'slices':6})
        results.append({'regime':regime['name'],'strategy':'splitting','executed':executed,'avg_price':avg_price})

        # Adaptive
        eng2 = MarketEngine(init_price=100.0, vol=0.0, rng=rng)
        for p, s in regime['initial_asks']:
            eng2.place_order({'side': 'sell', 'price': p, 'size': s, 'bot': None})
        eng2.register_bot(mm)
        ab = AdaptiveSplittingBot(aggressiveness=0.5, min_slice=0.1, max_slice=2.0)
        executed2, avg_price2 = run_strategy(eng2, ab, {'side':'buy','total_size':6.0})
        results.append({'regime':regime['name'],'strategy':'adaptive','executed':executed2,'avg_price':avg_price2})

        # Greedy
        eng3 = MarketEngine(init_price=100.0, vol=0.0, rng=rng)
        for p, s in regime['initial_asks']:
            eng3.place_order({'side': 'sell', 'price': p, 'size': s, 'bot': None})
        eng3.register_bot(mm)
        g = GreedyAdaptiveBot()
        executed3, avg_price3 = run_strategy(eng3, g, {'side':'buy','total_size':6.0})
        results.append({'regime':regime['name'],'strategy':'greedy','executed':executed3,'avg_price':avg_price3})

# Save results and plot
import csv
csv_path = ARTIFACTS / 'execution_benchmark.csv'
with open(csv_path, 'w', newline='') as f:
    w = csv.writer(f)
    w.writerow(['regime','strategy','executed','avg_price'])
    for r in results:
        w.writerow([r['regime'], r['strategy'], r['executed'], r['avg_price']])

# Simple plot: avg_price by regime and strategy
import pandas as pd
df = pd.DataFrame(results)
fig, ax = plt.subplots(figsize=(8,4))
for name, grp in df.groupby('regime'):
    ax.clear()
    sub = grp.pivot(index='strategy', columns='regime', values='avg_price')
    # easier: plot manually
    strategies = ['splitting','adaptive','greedy']
    prices = [grp[grp['strategy']==s]['avg_price'].mean() for s in strategies]
    ax.bar(strategies, prices, color=['C0','C1','C2'])
    ax.set_title(f'Average Exec Price — Regime {name}')
    ax.set_ylabel('Avg Exec Price')
    figpath = ARTIFACTS / f'exec_price_{name}.png'
    fig.tight_layout()
    fig.savefig(figpath)
    print('Saved', figpath)

print('Saved CSV to', csv_path)